# BikewaySim

Use this jupyter notebook to run BikewaySim. BikewaySim finds the shortest path between any origin and destination using Dijkstra's algorithm. 

BikewaySim requires the following before running:

- A nodes shapefile/geojson with node ids.
- A links shapefile/geojson with refrences ids (i.e. column that specifies the starting and ending node id)
- A csv with origin and destination pairs

Notes:
- Run the convert_to_bikewaysim_network script before running bikewaysim to make sure your network is compatible
- The desired link cost should be specifed in the "distance" column of the links. If you want to specify your own link costs using link attributes, then modify the "distance" column before using the network in BikewaySim

Other Notes:

Use Python 3.7+


The following third-party packages are used in the BikewaySim module:
```python
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

import networkx as nx

import matplotlib.pyplot as plt
```

# Preparation

Import modules/packages required for this notebook.

In [1]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpdui

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/%autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
pd.options.display.max_columns = None  # display all columns

In [2]:
# import TransportSim module
from core_script.getPaths_transportSim import *

### Setting the directory

In [3]:
# need to set this environmental path for network data and query data at separate locations
user_directory = os.fspath(Path.home()) #get home directory and convert to path string

# set path variable for BikewaySim
os.environ['PROJ_LIB'] = user_directory + "/Documents/GitHub/BikewaySimDev"

# network shapefile data path directory
os.environ['bws_NETWORK'] = user_directory + "/Documents/BikewaySimData/processed_shapefiles/prepared_network/imp_dist"
#os.environ['bws_NETWORK'] = user_directory + r"/Documents/BikewaySimData/processed_shapefiles/prepared_network/per_dist"

# OD Pairs directory and location of output files
os.environ['bws_TRIPS'] = user_directory + "/Documents/BikewaySimData/bikewaysim_outputs"

# create BikewaySim object for running 'bikewaysim' option
bws = BikewaySim(os.environ['PROJ_LIB'], os.environ['bws_NETWORK'], os.environ['bws_TRIPS'])

Initialize BikewaySim object to process sidewalk graph and pre-process trip queries!
Set/Update BikewaySim dict_settings!


### Prepare networks for BikewaySim

In [4]:
# prepare networks
df_links, dict_bike = bws.prepare_network(grid_size=10000)
# print default settings

** Initialize BikewaySim network **
It takes 20.273 seconds to run the function 'prepare_network()'!


In [5]:
# strategy: 1. given origin time find earliest arrival
#           2. given expected arrival time find latest departure time
dict_settings = {'walk_speed': 2.0,  # people's walking speed 2 mph
                 'grid_size': 10000.0,  # for searching nearby links by grouping links to grids with width 10000 ft. for efficiency in searching
                 'ntp_dist_thresh': 5280.0,  # node to point (maximum distance access to network from origin/destination); (walking) distance threshold
                 'network': {'bike': dict_bike},  # dump in networks and modes
                 # strategy determines network link's direction.
                 # Strategy 1: Find earliest arrival given query time as departure time
                 # Strategy 2: Find latest departure time given query time as arrival time
             
                 'strategy': {'bike': 1},  # 1. find earliest arrival 2. find latest departure
                 'query_time': [8],  # departure time or arrival time of a trip, depends on the strategy
                 
                 'walk_thresh': {'bike': 0},  # walking threshold is 0.3 miles
                 'num_options': {'bike': 1},  # if set to 2, return 2-shortest paths
                 'plot_all': False,  # if True, plot results and save plots for all routes found #this is broken now
                 'one_by_one': False  # set time and strategy one by one
                 }
# load dict_settings to the sws object
bws.dict_settings = dict_settings

Set/Update BikewaySim dict_settings!


In [6]:
df_points = bws.prepare_trips(option='bike')
display(df_points.head())

** load trip data & prepare sample **
It takes 1.205 seconds to run the function 'prepare_trips()'!


,trip_id,ori_id,dest_id,ori_lat,ori_lon,dest_lat,dest_lon,o_node,ox,oy,o_d,d_node,dx,dy,d_d,ox_sq,oy_sq,dx_sq,dy_sq,o_t,d_t
0,480_481,480,481,33.777463,-84.4042,33.774730,-84.405087,2044571610,2.224395e+06,1.374096e+06,74.71749,2044571520,2.224244e+06,1.373249e+06,130.433653,222.0,137.0,222.0,137.0,0.007076,0.012352
1,480_483,480,483,33.777463,-84.4042,33.772296,-84.399644,2044571610,2.224395e+06,1.374096e+06,74.71749,201181672273,2.225952e+06,1.372295e+06,165.595763,222.0,137.0,223.0,137.0,0.007076,0.015681
2,480_484,480,484,33.777463,-84.4042,33.775002,-84.397063,2044571610,2.224395e+06,1.374096e+06,74.71749,201181686534,2.226526e+06,1.373342e+06,86.228389,222.0,137.0,223.0,137.0,0.007076,0.008166
3,480_485,480,485,33.777463,-84.4042,33.780075,-84.400795,2044571610,2.224395e+06,1.374096e+06,74.71749,20908906190,2.225452e+06,1.375214e+06,96.719013,222.0,137.0,223.0,138.0,0.007076,0.009159
4,480_486,480,486,33.777463,-84.4042,33.779451,-84.394831,2044571610,2.224395e+06,1.374096e+06,74.71749,20311926081,2.227070e+06,1.374701e+06,262.146369,222.0,137.0,223.0,137.0,0.007076,0.024824


In [7]:
# run all queries
bws.run_all()

**for each trip, find k-shortest paths**
072622_115445
PATH #: 0
['2044571610', '2044571611', '20898953274', '20898953280', '20898927216', '20898927215', '20898927218', '20898927219', '20898953291', '20898927236', '20898927238', '20908741774', '20908741260', '20908741320', '20908743012', '20908743013', '20908741001', '20898927213', '20898927211', '20898927199', '201181686534']
PATH #: 0
['2044571610', '20908327253', '201181581154', '201181581152', '2044571615', '20908468109', '20908468108', '20908327260', '20908327257', '20908468079', '20908327256', '2044571618', '20908328476', '2044571617', '2044571680', '2044571623', '20908467707', '20908467708', '20908455882', '201181581144', '201181581145', '201181581146', '20979683964', '2044571688', '20908328655', '2044571689', '201181581139', '201181582054', '2044571624', '20908906190']
PATH #: 0
['2044571610', '20908327253', '201181581154', '201181581152', '2044571615', '20908468109', '20908468108', '20908327260', '20908468103', '20908327261', 

PATH #: 0
['201181686534', '20898927199', '20898927211', '20898927213', '20908741001', '20908743013', '20908743012', '20908741320', '20908741260', '20908741774', '20898927238', '20898927236', '20898953291', '20898927219', '20898927218', '20898927215', '20898927216', '20898953289', '20908743109', '20898953283', '2044571620', '201181582617', '201181582595', '20979683888', '2044571618', '20908328476', '2044571617', '201181581136', '20908327933', '2044571676', '20916267251', '201181583057', '2044571677', '2044571678', '2044571684', '2044571647']
PATH #: 0
['201181686534', '201181686533', '201181686536', '201181686535', '20898956743', '20908739514', '20908740495', '20908740498', '20908739516', '20908739878', '20908714081', '20908714082', '201181686105', '201181686101', '20908742935', '2044579851', '20801201491', '20908710011', '20908710550', '2044579852', '201181582053', '2044579946', '201181592187', '201181592191', '2044579947', '2044579948', '2044579951', '2044579952', '201181664701', '20

PATH #: 0
['201181686534', '201008511452', '201008511401', '20908738270', '20908738272', '20908743277', '20908743276', '20898956750', '20898956724', '20898956718', '20898956716', '2044579915', '2044579920', '201051471934', '2044579928', '20824589896', '2044579931', '2044580061', '201181570784', '201181570783', '20908608976', '20908608977', '2044580064', '2044580076', '2044580078', '201181591760', '201169402895', '2044580109', '2044580111', '2044580115', '2044580108']
PATH #: 0
['201181686534', '201008511452', '20898927188', '20898927192', '20908743049', '20908743050', '20908743052', '20908743053', '20908742072', '20908742077', '20908742078', '20908742084', '20908742083', '20908742081', '20908742075', '20908742076', '20898927247', '20898927248', '20898927245', '201181672290', '201181672292', '20898927242', '20898927241', '20907839041', '20907970582', '20907839042', '20907970905', '20907970906', '20879230842', '20907944386', '2044579890', '20816942147', '20816942148', '20850820499', '208

PATH #: 0
['20908906190', '2044571624', '201181582054', '201181581139', '2044571689', '2044571694', '20979670980', '20908705836', '2044571695', '201181582045', '201181582047', '20908704424', '20908704425', '20908743067', '20908743066', '2044579852', '20979683887', '20908710552', '20908710550', '201008917848', '20908742721', '20908742722', '20908710551', '2044579855', '20908743239', '20908743240', '20898954670', '20898954675', '20898954669', '20898954671', '20898954676', '20908736887', '20908736888', '20908736889', '20908736891', '2045978958', '20979683889', '20898956716', '2044579915', '2044579920', '201051471934', '2044579928', '20824589896', '2044579931', '201051471938', '201170253080', '201181663692', '2044579912', '2044580046', '2044580045', '201181570751', '201181570764', '201181570762']
PATH #: 0
['20908906190', '2044571624', '201181582054', '201181581139', '2044571689', '2044571694', '20979670980', '20908705836', '2044571695', '201181582045', '201181582047', '20908704424', '2090

PATH #: 0
['20908906190', '2044571624', '201181582054', '201181581139', '2044571689', '2044571694', '20979670980', '20908705836', '2044571695', '201181582045', '201181582047', '20908704424', '20908704425', '20908743067', '20908743066', '2044579852', '20979683887', '20908710552', '20908710550', '201008917848', '20908742721', '20908742722', '20908710551', '2044579855', '20908743239', '20908743240', '20898954670', '20898954675', '20898954669', '20898954671', '20898954676', '20908736887', '20908736888', '20908736889', '20908736891', '2045978958', '20979683889', '20898956716', '2044579915', '2044579920', '201051471934', '2044579928', '20824589896', '2044579931', '2044580061', '201181570784', '201181570783', '20908608976', '2044580065', '20979762439', '2044580066', '2044580077', '2044580080', '2044580082', '201188210198']
PATH #: 0
['20908906190', '2044571624', '201181582054', '201181581139', '2044571689', '2044571694', '20979670980', '20908705836', '2044571695', '201181582045', '20118158204

...

PATH #: 0
['20333830314', '20333830308', '20333830312', '20309438434', '20979715710', '2044584834', '20979806196', '201125277959', '2044584832', '2044584768', '201151035900', '2044584763', '2044584574', '2044584567', '20979806192', '201053322401', '2044584553', '2044584552', '2086088540', '201112862838', '201181582219', '2044584488', '2044584482', '20979806193', '20979806194', '201157256529', '2044581946', '2044581939', '201157663548', '2044581933', '2044581929', '201157742156', '20970108807', '20970108808', '2044581980', '2044581960', '2044581959', '2044581859', '20979757291', '20979821262', '2044581770', '2044581769', '2044581761', '20979742727', '2044581757', '201098698989', '2044581609', '2044581607', '201181587771', '2044581603', '20979742725', '2044581567', '201181587772', '2044581568', '2044581566', '2044580794', '2044580924', '2044580920', '2044580779']
PATH #: 0
['20333830314', '20333830308', '20333830312', '20309438434', '20979715710', '2044584834', '20979806196', '2011252779

PATH #: 0
['20333830314', '20333830308', '20333830312', '20309438434', '20979715710', '2044584834', '20979806196', '201125277959', '2044584832', '2044584768', '201151035900', '2044584763', '2044584727', '2044584550', '2044584532', '2044584467', '2044584465', '2044584460']
PATH #: 0
['20333830314', '20333830308', '20333830312', '20309438434', '20979715710', '2044584834', '20979806196', '201125277959', '2044584832', '2044584768', '201151035900', '2044584763', '2044584727', '2044584726', '2044584548']
PATH #: 0
['20333830314', '20333830308', '20333830304', '20333830301', '2044584821', '2044584427', '201181635352', '2044584437', '2044584433', '201181637634', '20980227851', '201165760720', '201181637655', '2044584412', '201181637656', '2044584369', '2044584339', '2044584335', '2044584136', '2044584135', '2044584134', '2044584122']
PATH #: 0
['20333830314', '20333830308', '20333830304', '20333830301', '2044584751', '201059097830', '2044584746', '201059097829', '2044584376', '201181583756', '

PATH #: 0
['20333830314', '20333830308', '20333830312', '20309438434', '20979715710', '2044584834', '20979806196', '201125277959', '2044584832', '2044584768', '201151035900', '2044584763', '2044584793', '2044584795', '2044584799', '2044584878']
PATH #: 0
['20333830314', '20333830308', '20333830312', '20309438434', '201157596776', '2044584830', '201157597837', '201157599221', '20851556517', '2044584859', '201181635305', '20809919045', '2044584862', '20809919044', '20835970511', '20835970512', '201165343557', '2044585463', '2086098032', '20306113955', '20306113956', '20306113971']
PATH #: 0
['20333830314', '20333830308', '20333830304', '20333830301', '2044584821', '2044584427', '201181635352', '2044584437', '2044584433', '201181637634', '20980227851', '201165760720', '201165760722', '201165760734', '201165760724', '201165760728', '201181637637', '201181637641', '201181637643']
PATH #: 0
['20333830314', '20333830308', '20333830304', '20333830301', '2044584821', '2044584427', '201181635352

PATH #: 0
['201181637643', '201181637641', '201181637637', '201165760728', '201165760724', '201165760734', '201165760722', '201165760720', '20980227851', '201181637634', '2044584433', '2044584437', '201181635352', '2044584427', '2044584821', '20333830301', '2044584751', '2044584752', '2045997836', '2044584832', '2044584768', '201151035900', '2044584763', '2044584574', '2044584567', '201140656418', '2044584568', '2044584569', '2044584573', '2044584582', '201063932694', '201156473553', '201063932693', '2044584561', '201112681963', '20946086648', '2044584645', '2044584644', '2044584604', '2044584593', '2044582081', '2044584597']
PATH #: 0
['201181637643', '201181637641', '201181637637', '201165760728', '201165760724', '201165760734', '201165760722', '201165760720', '20980227851', '201181637634', '2044584433', '2044584437', '201181635352', '2044584427', '2044584821', '20333830301', '2044584751', '2044584752', '2045997836', '2044584832', '2044584768', '201151035900', '2044584763', '20445847

PATH #: 0
['201181637643', '201181637641', '201181637637', '201165760728', '201165760724', '201165760734', '201165760722', '201165760720', '201181637655', '2044584412', '201181637656', '2044584369', '2044584339', '2044584335', '2044584136', '2044584135', '2044584121', '2044583998', '201181643835']
PATH #: 0
['201181643835', '2044583998', '2044583997', '2044584111', '20801777923', '2044584112', '2044584118', '2044584122', '2044584126', '2044584158', '2044584165', '2044584166', '2044584516', '2044584514', '2044584513', '20969621379', '201168245279', '20311897357', '20311897356', '2030638844', '20979791871', '2044584528', '2044584530', '2044584531', '2044584532', '20979806191', '20805320876', '2044584567', '201140656418', '2044584568', '2044584569', '2044584573', '2044584582', '201063932694', '201156473553', '201063932693', '2044584561', '201112681963', '20946086648', '2044584645', '201063936460', '201112682557', '201112682558']
PATH #: 0
['201181643835', '2044583998', '2044583997', '2044

# Results

The trip inputs & results are stored in ' ***/trips_bws/results*** '

### Columns defintion of the output results
Every row correpsonds to a link<br>
**A**: origin node id<br>
**B**: destination node id<br>
**dist**: distances<br>
**mode**: mode of transportation (e.g., walk, bike, etc.)<br>
**strategy**: chosen strategy<br>
**route_num**: the k number of k-shortest routes return for each link<br>
**sequence**: the trip segment number in a sequence of trips<br>
**time**: time it takes to travel through the link<br>
**timeStamp**: accumulative time of the current trip (in hours)<br>
**trip_id**: trip id (from the trip query)<br>
**route**: if there is a crosswalk, which crosswalk it uses<br>

### For graph outputs

***BikeSim/build_graph/results_routes***

naming rule:

{trip_id}_{(k-1)th route}.PNG

e.g.,

{4001880_1_1}_{0}.PNG